In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import SimpleITK as sitk
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from tqdm import tqdm
import shutil
import gc

# Paths and Config
ROOT_DICOM = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"
#OUTPUT_PNG_ROOT = "/kaggle/working/converted_png_224x224"  # temp directory for resized PNGs
ROOT_MASK = "/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations"
OUTPUT_PNG_ROOT = "/kaggle/tmp/converted_png_224x224"
OUTPUT_DIR = "/kaggle/tmp/resampled_output"

TARGET_SIZE = (224, 224)
TARGET_SPACING = (1.0, 1.0, 1.0)

os.makedirs(OUTPUT_PNG_ROOT, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load series to modality mapping CSV
metadata_csv = "/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv"
df = pd.read_csv(metadata_csv)
series_to_modality = dict(zip(df['SeriesInstanceUID'], df['Modality']))
modalities_to_include = ["CTA", "MRA", "MRI T1post", "MRI T2"]


In [2]:
def dicom_to_resized_png(dicom_dir, out_dir, target_size=TARGET_SIZE):
    os.makedirs(out_dir, exist_ok=True)
    dicom_files = sorted(glob.glob(os.path.join(dicom_dir, "*.dcm")))
    if not dicom_files:
        return False
    for i, dcm_file in enumerate(dicom_files):
        try:
            ds = pydicom.dcmread(dcm_file)
            img_array = ds.pixel_array.astype(np.float32)
            img_norm = (img_array - img_array.min()) / (img_array.max() - img_array.min() + 1e-8)
            img_uint8 = (img_norm * 255).astype(np.uint8)
            img_pil = Image.fromarray(img_uint8).convert("L")
            img_resized = img_pil.resize(target_size, Image.LANCZOS)
            img_resized.save(os.path.join(out_dir, f"{i:04d}.png"))
        except Exception as e:
            print(f"Failed to convert {dcm_file}: {e}")
            return False
    return True


In [3]:
def stack_png_series(series_dir, target_size=TARGET_SIZE):
    files = sorted(glob.glob(os.path.join(series_dir, "*.png")))
    slices = []
    for file in files:
        img = Image.open(file).convert("L")
        img = img.resize(target_size)
        arr = np.array(img, dtype=np.float32) / 255.0
        slices.append(arr)
    if not slices:
        return None
    return np.stack(slices, axis=0)

def np_volume_to_sitk(volume, spacing):
    img = sitk.GetImageFromArray(volume.astype(np.float32))
    img.SetSpacing(spacing)
    return img

def resample_sitk_img(sitk_img, new_spacing=TARGET_SPACING, is_label=False):
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    original_size = sitk_img.GetSize()
    original_spacing = sitk_img.GetSpacing()
    new_size = [
        int(round(osz * ospc / nspc))
        for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)
    ]
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(sitk_img.GetDirection())
    resampler.SetOutputOrigin(sitk_img.GetOrigin())
    interpolator = sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear
    resampler.SetInterpolator(interpolator)
    return resampler.Execute(sitk_img)

def get_dicom_spacing(dicom_dir):
    dicom_files = [f for f in os.listdir(dicom_dir) if f.endswith(".dcm")]
    if not dicom_files:
        raise RuntimeError(f"No DICOM files found in {dicom_dir}")
    for dcm_file in dicom_files:
        dcm = pydicom.dcmread(os.path.join(dicom_dir, dcm_file), stop_before_pixels=True)
        if hasattr(dcm, "SliceThickness") and hasattr(dcm, "PixelSpacing"):
            z = float(dcm.SliceThickness)
            y, x = [float(v) for v in dcm.PixelSpacing]
            return (z, y, x)
    raise RuntimeError(f"No valid DICOM with SliceThickness and PixelSpacing found in {dicom_dir}")


In [4]:
def process_case(args):
    artery, series, modality, ROOT_PNG, ROOT_MASK, ROOT_DICOM, OUTPUT_DIR, target_spacing = args
    series_dir = os.path.join(ROOT_PNG, modality, series)
    dicom_dir = os.path.join(ROOT_DICOM, series)
    out_dir_mod = os.path.join(OUTPUT_DIR, modality)
    os.makedirs(out_dir_mod, exist_ok=True)
    out_vol_path = os.path.join(out_dir_mod, f"{series}_vol_resamp.nii.gz")
    out_mask_path = os.path.join(out_dir_mod, f"{series}_mask_resamp.nii.gz")
    try:
        orig_spacing = get_dicom_spacing(dicom_dir)
        vol_np = stack_png_series(series_dir)
        if vol_np is None:
            return f"{series} [{modality}] FAIL: No PNG slices found"
        vol_sitk = np_volume_to_sitk(vol_np, spacing=orig_spacing)
        vol_resamp = resample_sitk_img(vol_sitk, new_spacing=target_spacing, is_label=False)
        sitk.WriteImage(vol_resamp, out_vol_path)
        mask_path = os.path.join(ROOT_MASK, f"{series}.nii")
        if os.path.exists(mask_path):
            mask_sitk = sitk.ReadImage(mask_path)
            mask_resamp = resample_sitk_img(mask_sitk, new_spacing=target_spacing, is_label=True)
            sitk.WriteImage(mask_resamp, out_mask_path)
        return f"{series} [{modality}] SUCCESS"
    except Exception as e:
        return f"{series} [{modality}] FAIL: {e}"


In [5]:
def batch_process(series_info):
    series, modality, dicom_dir = series_info
    out_dir = os.path.join(OUTPUT_PNG_ROOT, modality, series)
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    success = dicom_to_resized_png(dicom_dir, out_dir, TARGET_SIZE)
    if not success:
        return f"{series} [{modality}] Conversion failed"
    artery = ""  # Add artery info if available
    args = (artery, series, modality, OUTPUT_PNG_ROOT, ROOT_MASK, ROOT_DICOM, OUTPUT_DIR, TARGET_SPACING)
    result = process_case(args)
    shutil.rmtree(out_dir)
    gc.collect()
    return result


In [ ]:
series_list = [
    (series, modality, os.path.join(ROOT_DICOM, series))
    for series, modality in series_to_modality.items()
    if modality in modalities_to_include and os.path.isdir(os.path.join(ROOT_DICOM, series))
]

batch_size = 10
max_workers = 5

from tqdm import tqdm
from concurrent.futures import as_completed, ProcessPoolExecutor

total_files = 4348  # Total number of DICOM series

with tqdm(total=total_files, desc="Total Resampling Progress") as pbar:
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(batch_process, si) for si in series_list]
        
        # as each future completes update the main progress bar by 1
        for future in as_completed(futures):
            result = future.result()
            pbar.update(1)
            print(result)


print("✅ Full pipeline complete.")


Total Resampling Progress:   0%|          | 0/4348 [00:00<?, ?it/s]

Failed to convert /kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657/1.2.826.0.1.3680043.8.498.75206494637570575939256404615022232157.dcm: Cannot handle this data type: (1, 1, 528), |u1


Total Resampling Progress:   0%|          | 1/4348 [00:05<6:49:05,  5.65s/it]

1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657 [MRA] Conversion failed


Total Resampling Progress:   0%|          | 2/4348 [00:10<6:08:56,  5.09s/it]

...

1.2.826.0.1.3680043.8.498.80898851576943285501253651487538010386 [CTA] SUCCESS


Total Resampling Progress:  83%|████████▎ | 3617/4348 [3:55:15<28:21,  2.33s/it]

1.2.826.0.1.3680043.8.498.81132453359794713663612665742839717421 [MRI T2] SUCCESS


Total Resampling Progress:  83%|████████▎ | 3618/4348 [3:55:23<46:10,  3.80s/it]

1.2.826.0.1.3680043.8.498.81181812554261337166445969981198673971 [CTA] SUCCESS
